In [1]:
import matplotlib.pyplot as plt
import pickle
import os
import snap
from itertools import combinations
import numpy as np
from google.cloud import bigquery
import collections
from IPython.lib import backgroundjobs as bg
from tqdm import tqdm_notebook
import heapq
from gensim.models import Word2Vec
import node2vec
import networkx as nx
from sklearn import metrics

In [2]:
with open('./data/2016_q1_cci-neighbours_kcore.pickle', 'rb') as picklefile:
    neighbours_q1 = pickle.load(picklefile)

In [3]:
with open('./data/2016_q1_cci-weights_kcore.pickle', 'rb') as picklefile:
    weights_q1 = pickle.load(picklefile)

In [18]:
dimensions=32
window=10
iterations=10
p=0.25
q=4
num_walks=25
walk_length = 400

In [5]:
nx_weighted = nx.Graph()
nx_weighted.add_nodes_from(neighbours_q1.keys())
for edge in weights_q1:
    u, v = edge[0]
    nx_weighted.add_edge(u, v, weight=edge[1])

In [6]:
G = node2vec.Graph(nx_weighted, False, p, q)
%time G.preprocess_transition_probs()

CPU times: user 32min 58s, sys: 12 s, total: 33min 10s
Wall time: 33min 12s


In [7]:
%time walks = G.simulate_walks(num_walks, walk_length)
walks = [map(str, walk) for walk in walks]
%time model = Word2Vec(walks, size=dimensions, window=window, min_count=0, sg=1, iter=iterations)

Walk iteration:
1 / 25
2 / 25
3 / 25
4 / 25
5 / 25
6 / 25
7 / 25
8 / 25
9 / 25
10 / 25
11 / 25
12 / 25
13 / 25
14 / 25
15 / 25
16 / 25
17 / 25
18 / 25
19 / 25
20 / 25
21 / 25
22 / 25
23 / 25
24 / 25
25 / 25
CPU times: user 59min 54s, sys: 47.7 s, total: 1h 42s
Wall time: 1h 2min 4s
CPU times: user 28min 7s, sys: 4.46 s, total: 28min 12s
Wall time: 9min 27s


In [8]:
with open('./data/2016_q1_cci-weights_kcore.pickle', 'rb') as picklefile:
    weights_q1 = pickle.load(picklefile)
with open('./data/2016_q2_cci-weights_kcore.pickle', 'rb') as picklefile:
    weights_q2 = pickle.load(picklefile)
edges_q1 = set(edge for edge, weight in weights_q1)
edges_q2 = set(edge for edge, weight in weights_q2)
new_edges=edges_q2.difference(edges_q1)
k = len(new_edges)

In [9]:
authors = project_authors_q1['author_id'].unique()
pairs = list(combinations(authors, 2))
dist = []
for u, v in tqdm_notebook(pairs, mininterval=1):
    if u > v:
        u, v = v, u
    if (u, v) not in edges_q1:
        dist.append((model.wv.distance(str(u), str(v)), (u, v)))
dist = sorted(dist)
topk = set(edge for weight, edge in dist[:k])
len(topk.intersection(new_edges)), len(topk.intersection(new_edges))/float(k)

NameError: name 'project_authors_q1' is not defined

100, 25 (2864, 0.11310323039254404)
100, 100 (3459, 0.1282488598865448)

In [19]:
len(neighbours_q1.keys())

3053

In [10]:
authors = neighbours_q1.keys()
pairs = list(combinations(authors, 2))
dist = []
for u, v in tqdm_notebook(pairs, mininterval=1):
    if u > v:
        u, v = v, u
    if (u, v) not in edges_q1:
        dist.append((model.wv.distance(str(u), str(v)), (u, v)))
dist = sorted(dist)
topk = set(edge for weight, edge in dist[:k])
len(topk.intersection(new_edges)), len(topk.intersection(new_edges))/float(k)

/Users/yilingc/Courses/CS224w/project/GithubNetworkAnalysis/snap-2.7/env2.7/lib/python2.7/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


(4260, 0.15794742501204997)

In [16]:
import random
candidates = list(set(pairs).difference(edges_q1))
len(set(random.sample(candidates, k)).intersection(new_edges))

63

In [17]:
4260/63

67

In [15]:
help(random.sample)

Help on method sample in module random:

sample(self, population, k) method of random.Random instance
    Chooses k unique random elements from a population sequence.
    
    Returns a new list containing elements from the population while
    leaving the original population unchanged.  The resulting list is
    in selection order so that all sub-slices will also be valid random
    samples.  This allows raffle winners (the sample) to be partitioned
    into grand prize and second place winners (the subslices).
    
    Members of the population need not be hashable or unique.  If the
    population contains repeats, then each occurrence is a possible
    selection in the sample.
    
    To choose a sample in a range of integers, use xrange as an argument.
    This is especially fast and space efficient for sampling from a
    large population:   sample(xrange(10000000), 60)



In [20]:
len(project_authors_q1['author_id'].unique())

3053

In [24]:
pairs = list(combinations(authors, 2))
alldist = {}
for u, v in tqdm_notebook(pairs, mininterval=1):
    if u > v:
        u, v = v, u
    alldist[(u, v)] = model.wv.distance(str(u), str(v))
with open('./data/node2vec-weighted-p10-q01-100-100.pickle', 'wb') as picklefile:
    pickle.dump(alldist, picklefile)

In [11]:
authors = neighbours_q1.keys()
pairs = list(combinations(authors, 2))
truth = [edge in edges_q2 for edge in pairs]
scores = [model.wv.similarity(str(u), str(v)) for u, v in pairs]
fpr, tpr, thres = metrics.roc_curve(truth,scores)
metrics.auc(fpr,tpr), metrics.average_precision_score(truth,scores)

(0.9294000348665274, 0.16941554266646455)

In [27]:
sum(truth)

99311

In [19]:
pairs = [(u, v) if u < v else (v, u) for u, v in combinations(authors, 2)]
minus_q2 = list(set(pairs).difference(edges_q2))
y_train = [edge in edges_q1 for edge in minus_q2]
x_train = [abs(model.wv.get_vector(str(u))- model.wv.get_vector(str(v))) for u, v in minus_q2]
edgerep = [abs(model.wv.get_vector(str(u))- model.wv.get_vector(str(v))) for u, v in pairs]
from sklearn.linear_model import LogisticRegression
lg = LogisticRegression(penalty='l2',C=0.001)
lg.fit(x_train,y_train)
lg_y_pred_est = lg.predict_proba(edgerep)[:,1]
fpr,tpr,thresholds = metrics.roc_curve(truth,lg_y_pred_est)
metrics.auc(fpr,tpr), metrics.average_precision_score(truth,lg_y_pred_est)

/Users/yilingc/Courses/CS224w/project/GithubNetworkAnalysis/snap-2.7/env2.7/lib/python2.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


(0.937317540772617, 0.18081478319279892)